In [ ]:
import xdggs
import xarray as xr
import numpy as np

ds = xdggs.tutorial.open_dataset("air_temperature", "healpix").dggs.decode()
ds

In [ ]:
def as_cf(ds):
    grid_info = ds.dggs.grid_info
    metadata = {
        "grid_mapping_name": "healpix",
        "refinement_level": grid_info.level,
        "indexing_scheme": grid_info.indexing_scheme,
        "reference_body": "WGS84",
        "refinement_ratio": 4,
    }

    new = ds.assign_coords(crs=xr.Variable((), np.int8(0), metadata))
    new["cell_ids"].attrs = {"standard_name": "healpix", "units": "1"}

    var_attrs = {"coordinates": ds.dggs._name, "grid_mapping": "crs"}

    for name, var in new.variables.items():
        if name in {"cell_ids"} or "cells" not in var.dims:
            continue

        var.attrs |= var_attrs

    return new

In [ ]:
ds.pipe(as_cf)